In [76]:
# Import Required Packages
import pandas as pd
import geopandas as gpd
import csv
from csv import writer
import requests
import json
import numpy as np
import time
import plotly.graph_objects as go
import plotly.offline as go_offline

In [87]:
#Read shapefile and get coordinate bounds
vector_file_path=r"C:\Users\Matthew Carter\Downloads\Yosemite\CA.shp"
vector_data=gpd.read_file(vector_file_path)
vector_bound_coordinates= vector_data['geometry']
minimumx = vector_bound_coordinates.bounds.minx
minimumy = vector_bound_coordinates.bounds.miny
maximumx = vector_bound_coordinates.bounds.maxx
maximumy = vector_bound_coordinates.bounds.maxy
print(minimumx, maximumx, minimumy, maximumy)

0   -124.409591
Name: minx, dtype: float64 0   -114.131211
Name: maxx, dtype: float64 0    32.534156
Name: miny, dtype: float64 0    42.009485
Name: maxy, dtype: float64


In [88]:
sq_num = 50
x_range = np.linspace(start=minimumx, stop=maximumx, num=sq_num)
y_range = np.linspace(start=minimumy, stop=maximumy, num=sq_num)
y_range_list = y_range.tolist()
x_range_list = x_range.tolist()

list

In [89]:
#create list of x and y values stitched together
xy_list = []
for x in x_range:
    for y in y_range:
        yx_val = str(y)+str(x)+"|"
        yx_val_rep = yx_val.replace("[","").replace("][-","-").replace("]","")
        xy_list.append(yx_val_rep)
        
#turn xy values into a string for passing through the API
xy_str=str(xy_list)
xy_str_stripped = xy_str.replace("'","").replace(", ","").replace("[","").replace("]","").replace("-",",-")

requests_count = xy_str_stripped.count('|')/sq_num
print(requests_count)

50.0


In [80]:
#define function for finding position of character
def find_nth_overlapping(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+1)
        n -= 1
    return start

In [81]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [82]:
API_Count = 0
K = "|"
N = sq_num
z_vals = []
position = 0
first_position = 0
xy_str_stripped_copy = xy_str_stripped
while API_Count < sq_num:
    position = find_nth_overlapping(xy_str_stripped_copy,K,N)
    suffix = xy_str_stripped_copy[first_position:position]
    first_position = 0+position
    time.sleep(1)
    N=N+sq_num
    elevation_request = "https://api.opentopodata.org/v1/test-dataset?locations={}".format(suffix)
    r = requests.get(elevation_request)
    data = r.json()
    for i in data['results']:
        z_vals.append(i['elevation'])
    append_list_as_row('GA_coords.csv', z_vals)
    API_Count = API_Count+1
    z_vals = []
    print(API_Count)
print('API call complete')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
API call complete


In [98]:
# Read data from a csv
z_data = pd.read_csv('CA_Coords.csv')
fig = go.Figure(data=[go.Surface(z=z_data.values)])

fig.update_layout(title='California Elevation', autosize=False,
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90))
go_offline.plot(fig,filename='C:/Users/Matthew Carter/Downloads/California_Local.html',validate=True, auto_open=True)

'C:/Users/Matthew Carter/Downloads/California_Local.html'